id: household

Choices:  
- heinz41  
- heinz32  
- heinz28  
- hunts32  

variables  
- disp (1/0)  
- feat (1/0)  
- price (Log?)  

Running the model

In [6]:
from src.main import Mixedlogit, datasetloader 

dataset = datasetloader("n 1000 - seed 1234 - time 2019-06-20-134806.pickle")

mixedlogit = Mixedlogit(dataset, "bigbatch")

# #ndraws 50
mixedlogit.fit(drawtype="pseudo", n_draws=50, verbose=1, save="Base/parallel-1", start=0, end=200, parallel=True)
mixedlogit.fit(drawtype="pseudo", n_draws=50, verbose=1, save="Base/parallel-2", start=200, end=400, parallel=True)
mixedlogit.fit(drawtype="pseudo", n_draws=50, verbose=1, save="Base/parallel-3", start=400, end=600, parallel=True)
mixedlogit.fit(drawtype="pseudo", n_draws=50, verbose=1, save="Base/parallel-4", start=600, end=800, parallel=True)
mixedlogit.fit(drawtype="pseudo", n_draws=50, verbose=1, save="Base/parallel-5", start=800, end=1000, parallel=True)

# #ndraws 100
mixedlogit.fit(drawtype="pseudo", n_draws=100, verbose=1, save="Base/parallel-1", start=0, end=200, parallel=True)
mixedlogit.fit(drawtype="pseudo", n_draws=100, verbose=1, save="Base/parallel-2", start=200, end=400, parallel=True)
mixedlogit.fit(drawtype="pseudo", n_draws=100, verbose=1, save="Base/parallel-3", start=400, end=600, parallel=True)
mixedlogit.fit(drawtype="pseudo", n_draws=100, verbose=1, save="Base/parallel-4", start=600, end=800, parallel=True)
mixedlogit.fit(drawtype="pseudo", n_draws=100, verbose=1, save="Base/parallel-5", start=800, end=1000, parallel=True)

In [ ]:
from src.main import Mixedlogit, datasetloader 

dataset = datasetloader("n 1000 - seed 1234 - time 2019-06-20-134806.pickle")

mixedlogit = Mixedlogit(dataset, "bigbatch")


mixedlogit.fit(drawtype="pseudo", n_draws=250, verbose=1, save="Big/250", start=0, end=200, parallel=True)

mixedlogit.fit(drawtype="pseudo", n_draws=500, verbose=1, save="Big/500", start=0, end=200, parallel=True)

mixedlogit.fit(drawtype="pseudo", n_draws=1000, verbose=1, save="Big/1000", start=0, end=200, parallel=True)

mixedlogit.fit(drawtype="pseudo", n_draws=2000, verbose=1, save="Big/2000", start=0, end=200, parallel=True)

Fitting 200 datasets 

Executing code in parallel 



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 72 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 12.8min


In [1]:
from src.main import Mixedlogit, datasetloader 

dataset = datasetloader("n 1000 - seed 1234 - time 2019-06-20-134806.pickle")

mixedlogit = Mixedlogit(dataset, "bigbatch")

#ndraws 25
mixedlogit.fit(drawtype="pseudo", n_draws=25, verbose=1, save="Base/parallel-1", start=0, end=500, parallel=True)
mixedlogit.fit(drawtype="pseudo", n_draws=25, verbose=1, save="Base/parallel-2", start=500, end=1000, parallel=True)

#ndraws 75
mixedlogit.fit(drawtype="pseudo", n_draws=75, verbose=1, save="Base/parallel-1", start=0, end=500, parallel=True)
mixedlogit.fit(drawtype="pseudo", n_draws=75, verbose=1, save="Base/parallel-4", start=500, end=1000, parallel=True)

Fitting 500 datasets 

Executing code in parallel 



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 72 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 121 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done 169 tasks      | elapsed:  9.8min
[Parallel(n_jobs=-1)]: Done 194 tasks      | elapsed: 11.1min
[Parallel(n_jobs=-1)]: Done 221 tasks      | elapsed: 12.2min
[Parallel(n_jobs=-1)]: Done 248 tasks      | elapsed: 13.2min
[Parallel(n_jobs=-1)]: Done 277 tasks      | elapsed: 14.7min
[Parallel(n_jobs=-1)]: Done 306 tasks      | elapsed: 16.4min
[Parallel(n_jobs=-1)]: Done 337 tasks      | elapsed: 1

Finished parallel excuting of 500 jobs in 0:26:37, time per job 0:00:03

Initializing save
All results saved to pickle: Results/Base/parallel-1 - dgp bigbatch- start 0 end 499 - drawtype pseudo - ndraws 25- timestamp 2019-06-22-102601
Fitting 500 datasets 

Executing code in parallel 



[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 121 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done 169 tasks      | elapsed:  9.5min
[Parallel(n_jobs=-1)]: Done 194 tasks      | elapsed: 10.5min
[Parallel(n_jobs=-1)]: Done 221 tasks      | elapsed: 11.6min
[Parallel(n_jobs=-1)]: Done 248 tasks      | elapsed: 13.2min
[Parallel(n_jobs=-1)]: Done 277 tasks      | elapsed: 14.2min
[Parallel(n_jobs=-1)]: Done 306 tasks      | elapsed: 15.8min
[Parallel(n_jobs=-1)]: Done 337 tasks      | elapsed: 17.2min
[Parallel(n_jobs=-1)]: Done 408 out of 500 | elapsed: 20.3min remainin

Finished parallel excuting of 500 jobs in 0:25:59, time per job 0:00:03

Initializing save
All results saved to pickle: Results/Base/parallel-2 - dgp bigbatch- start 500 end 999 - drawtype pseudo - ndraws 25- timestamp 2019-06-22-105238
Fitting 500 datasets 

Executing code in parallel 



[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  7.4min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 10.3min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed: 11.8min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 14.2min
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed: 17.9min
[Parallel(n_jobs=-1)]: Done 121 tasks      | elapsed: 22.3min
[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed: 24.3min
[Parallel(n_jobs=-1)]: Done 169 tasks      | elapsed: 27.8min
[Parallel(n_jobs=-1)]: Done 194 tasks      | elapsed: 31.1min
[Parallel(n_jobs=-1)]: Done 221 tasks      | elapsed: 34.3min
[Parallel(n_jobs=-1)]: Done 248 tasks      | elapsed: 38.5min
[Parallel(n_jobs=-1)]: Done 277 tasks      | elapsed: 42.0min
[Parallel(n_jobs=-1)]: Done 306 tasks      | elapsed: 45.9min
[Parallel(n_jobs=-1)]: Done 337 tasks      | elapsed: 49.9min
[Parallel(n_jobs=-1)]: Done 408 out of 500 | elapsed: 58.4min remainin

Finished parallel excuting of 500 jobs in 1:15:43, time per job 0:00:09

Initializing save
All results saved to pickle: Results/Base/parallel-1 - dgp bigbatch- start 0 end 499 - drawtype pseudo - ndraws 75- timestamp 2019-06-22-111837
Fitting 500 datasets 

Executing code in parallel 



[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 75.7min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 72 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed: 12.0min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 14.2min
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed: 17.8min
[Parallel(n_jobs=-1)]: Done 121 tasks      | elapsed: 20.1min
[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed: 23.2min
[Parallel(n_jobs=-1)]: Done 169 tasks      | elapsed: 26.8min
[Parallel(n_jobs=-1)]: Done 194 tasks      | elapsed: 29.9min
[Parallel(n_jobs=-1)]: Done 221 tasks      | elapsed: 33.2min
[Parallel(n_jobs=-1)]: Done 248 tasks      | elapsed: 36.8min
[Parallel(n_jobs=-1)]: Done 277 tasks      | elapsed: 40.4min
[Parallel(n_jobs=-1)]: Done 306 tasks      | e

Finished parallel excuting of 500 jobs in 1:24:17, time per job 0:00:10

Initializing save
All results saved to pickle: Results/Base/parallel-4 - dgp bigbatch- start 500 end 999 - drawtype pseudo - ndraws 75- timestamp 2019-06-22-123421


[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 84.3min finished


In [ ]:
from src.main import Mixedlogit, datasetloader 

dataset = datasetloader("n 1000 - seed 1234 - time 2019-06-20-134806.pickle")

mixedlogit = Mixedlogit(dataset, "bigbatch")


# mixedlogit.fit(drawtype="halton", n_draws=25, verbose=1, save="Halton2/25-2", start=100, end=1000, parallel=True)

        
#ndraws pseudo 5
mixedlogit.fit(drawtype="pseudo", n_draws=5, verbose=1, save="Base/parallel-1", start=0, end=500, parallel=True)
mixedlogit.fit(drawtype="halton", n_draws=250, verbose=1, save="Halton2/250", start=0, end=200, parallel=True)

#make halton draws
partitions = 2
# for i in range(partitions):
i = 1
for r in [10, 50, 75, 100]:
    part = int(1000 / partitions)
    start = part * i
    end = part * (i+1)
    mixedlogit.fit(drawtype="halton", n_draws=r, verbose=1, save="Halton2/"+str(r)+"-"+str(i+1), start=start, end=end, parallel=True)

mixedlogit.fit(drawtype="pseudo", n_draws=5, verbose=1, save="Base/parallel-2", start=500, end=1000, parallel=True)

Fitting 500 datasets 

Executing code in parallel 



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 72 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   30.4s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 121 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 169 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 194 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 221 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 248 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 277 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done 306 tasks      | elapsed:  7.9min
[Parallel(n_jobs=-1)]: Done 337 tasks      | elapsed:  

Finished parallel excuting of 500 jobs in 0:12:33, time per job 0:00:02

Initializing save
All results saved to pickle: Results/Base/parallel-1 - dgp bigbatch- start 0 end 499 - drawtype pseudo - ndraws 5- timestamp 2019-06-23-183924
Fitting 200 datasets 

Executing code in parallel 



[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 10.3min
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed: 18.3min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 27.0min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed: 33.7min
[Parallel(n_jobs=-1)]: Done  78 out of 200 | elapsed: 42.8min remaining: 66.9min
[Parallel(n_jobs=-1)]: Done  99 out of 200 | elapsed: 49.0min remaining: 50.0min
[Parallel(n_jobs=-1)]: Done 120 out of 200 | elapsed: 57.9min remaining: 38.6min
[Parallel(n_jobs=-1)]: Done 141 out of 200 | elapsed: 68.0min remaining: 28.4min
[Parallel(n_jobs=-1)]: Done 162 out of 200 | elapsed: 72.8min remaining: 17.1min
[Parallel(n_jobs=-1)]: Done 183 out of 200 | elapsed: 79.9min remaining:  7.4min


Finished parallel excuting of 200 jobs in 1:39:07, time per job 0:00:30

Initializing save
All results saved to pickle: Results/Halton2/250 - dgp bigbatch- start 0 end 199 - drawtype halton - ndraws 250- timestamp 2019-06-23-185157
Fitting 500 datasets 

Executing code in parallel 



[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed: 99.1min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 72 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   38.5s
